In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from google.colab import drive

In [7]:
# Step 1: Extract the image data
drive.mount('/content/drive')

df_breast = pd.read_pickle('/content/breast_df2.dat')
df_lung = pd.read_pickle('/content/lung_df.pkl')
df_colon = pd.read_pickle('/content/colon_df.pkl')

MessageError: ignored

In [ ]:
# Concatenate the DataFrames vertically
merged_df = pd.concat([df_breast, df_lung, df_colon], axis=0)

# Reset the index of the merged DataFrame
merged_df = merged_df.reset_index(drop=True)

merged_df

In [ ]:
# Step 2: Preprocess the image data

# Convert the image data to NumPy arrays
image_data = np.array(merged_df['image'].tolist())

# Normalize the pixel values to range between 0 and 1
image_data = (image_data / 255.0) - 0.5

# Reshape the image data to have a single color channel (assuming it's RGB)
image_data = image_data.reshape(image_data.shape[0], image_data.shape[1], image_data.shape[2], 3)


In [ ]:
# Step 3: Convert the labels to categorical values

# Convert the labels to numerical values
label_mapping = {'breast': 0, 'colon': 1, 'lung': 2}
labels = merged_df['area'].map(label_mapping)

# Convert the numerical labels to one-hot encoded vectors
labels = keras.utils.to_categorical(labels, num_classes=3)


In [ ]:
# Step 4: Split the data into training and testing sets

# Split the data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(
    image_data, labels, test_size=0.2, random_state=42
)

In [ ]:
# Step 5: Build the CNN model
model = keras.Sequential([
    layers.Conv2D(8, (3, 3), activation='relu', input_shape=(image_data.shape[1], image_data.shape[2], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(16, (3, 3), activation='relu'),
    #layers.MaxPooling2D((2, 2)),
    #layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(8, activation='relu'),
    layers.Dense(3, activation='softmax')
])

In [ ]:
# Step 6: Compile and train the model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_images, train_labels, epochs=20, validation_data=(test_images, test_labels))

In [ ]:
# Step 7: Evaluate the model

test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

# Calculate additional metrics
predictions = model.predict(test_images)

# Apply threshold to obtain binary predictions
threshold = 0.5
binary_predictions = np.where(predictions >= threshold, 1, 0)

# Convert test_labels to 1D array
test_labels_1d = np.argmax(test_labels, axis=1)

# Convert predictions to 1D array
binary_predictions_1d = np.argmax(binary_predictions, axis=1)

# Confusion matrix
confusion = confusion_matrix(test_labels_1d, binary_predictions_1d)
print("Confusion Matrix:")
print(confusion)

# True positive (TP), true negative (TN), false positive (FP), false negative (FN)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

# Sensitivity, Recall, Hit Rate, or True Positive Rate (TPR)
sensitivity = TP / (TP + FN)
print(f'Sensitivity/Recall: {sensitivity}')

# Specificity or True Negative Rate (TNR)
specificity = TN / (TN + FP)
print(f'Specificity: {specificity}')

# Precision or Positive Predictive Value (PPV)
precision = TP / (TP + FP)
print(f'Precision: {precision}')

# Negative Predictive Value (NPV)
npv = TN / (TN + FN)
print(f'Negative Predictive Value: {npv}')

# F1 Score (Harmonic mean of precision and recall)
f1_score = 2 * (precision * sensitivity) / (precision + sensitivity)
print(f'F1 Score: {f1_score}')

# Matthews Correlation Coefficient (MCC)
mcc = (TP * TN - FP * FN) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
print(f'Matthews Correlation Coefficient: {mcc}')

# Recall
recall = sensitivity
print(f'Recall: {recall}')

# Receiver Operating Characteristic (ROC) Curve
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(3):  # Assuming you have 3 classes
    fpr[i], tpr[i], _ = roc_curve(test_labels[:, i], predictions[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curves
plt.figure()
colors = ['red', 'green', 'blue']  # Assuming you have 3 classes
for i in range(3):
    plt.plot(fpr[i], tpr[i], color=colors[i], label='ROC curve (area = %0.2f)' % roc_auc[i])

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Step 8: Plot accuracy and loss curves

# Plot training and validation accuracy
plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Guardar el modelo
model.save('./sample_data/body_part_model.h5')